In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
from os import sys
sys.path.append("/meshAfterParty/")

In [2]:
import datajoint_utils as du
import datajoint as dj

INFO - 2021-01-14 04:49:00,606 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-14 04:49:00,607 - settings - Setting database.user to celiib
INFO - 2021-01-14 04:49:00,608 - settings - Setting database.password to newceliipass
INFO - 2021-01-14 04:49:00,700 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-14 04:49:00,701 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-14 04:49:00,716 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-14 04:49:01,021 - settings - Setting enable_python_native_blobs to True


In [3]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-14 04:49:03,240 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-14 04:49:03,546 - settings - Setting enable_python_native_blobs to True


# Inspecting Deimation

In [17]:

"""864691135654122946
864691135583731320"""

minnie.Decimation() & dict(segment_id=864691135654122946)

segment_id segment id unique within each Segmentation,version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),n_vertices,n_faces,mesh in-place path to the hdf5 (decimated) mesh file
864691135654122946,0,0.25,1829353,3693265,=BLOB=


In [18]:
minnie.Decimation() & dict(segment_id=864691135583731320)

segment_id segment id unique within each Segmentation,version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),n_vertices,n_faces,mesh in-place path to the hdf5 (decimated) mesh file
864691135583731320,0,0.25,2710109,5474056,=BLOB=


In [19]:
minnie.Decimation() & dict(segment_id=864691135474614848)

segment_id segment id unique within each Segmentation,version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),n_vertices,n_faces,mesh in-place path to the hdf5 (decimated) mesh file
864691135474614848,0,0.25,2377488,4762293,=BLOB=


# Inspecting Baylor Soma

In [20]:
((schema.jobs & "table_name='__baylor_segment_centroid'"))#.delete()#& "status='error'" & "key_hash='d97066fa7288bd42364bbdc01f5062b7'").delete()#.fetch1("key"))

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__baylor_segment_centroid,002d1754f817477b388ae89754951316,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.156,at-node29,19,35921,2021-01-14 13:15:52
__baylor_segment_centroid,00465d10aae39f80a748ff5dc0b72437,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,19,36053,2021-01-14 13:23:52
__baylor_segment_centroid,018353e2df71045e714cbabcb84c61c6,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.129,at-node2,19,37997,2021-01-14 08:58:01
__baylor_segment_centroid,019f1f77b4a17e38e79ed3f6a1d16ea8,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.164,at-node37,20,37894,2021-01-14 13:16:04
__baylor_segment_centroid,02215f334440dbb048ba5c459e0f08c0,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.168,at-node41,24,38140,2021-01-14 13:24:53
__baylor_segment_centroid,0265f2033d616e19e8fae8012d27086c,error,=BLOB=,Exception: neuron 864691135688142102 did not fix the manifold edges (meshlab script failed) with output:,=BLOB=,celiib@10.28.0.83,at-compute004,19,36067,2021-01-14 02:17:03
__baylor_segment_centroid,04add0cdb154e0b5c05d5f0c7a9a4bd2,reserved,=BLOB=,,=BLOB=,celiib@10.66.4.64,jr-compute001,19,38126,2021-01-14 13:23:53
__baylor_segment_centroid,050f9b0280495daf2ef2f079a58c6721,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.152,at-node25,19,35977,2021-01-14 13:15:22
__baylor_segment_centroid,05be65a239aeb96c628ce7a3226a511b,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.148,at-node21,19,36903,2021-01-14 08:59:44
__baylor_segment_centroid,05c99f5329b111da88fb0f359f97252f,error,=BLOB=,Exception: neuron 864691135968871013 did not fix the manifold edges (meshlab script failed) with output:,=BLOB=,celiib@10.28.0.148,at-node21,19,36092,2021-01-14 08:01:37


In [ ]:
minnie

In [ ]:
key_source =  (((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}") & (dj.U("segment_id") & (minnie.OldBaylorSegmentCentroid() & "multiplicity<3").proj()) & minnie.NucleiSegmentsRun2() & dict(segment_id=864691135135964780))
                                                                 
     

In [26]:
decimation_version = 0
decimation_ratio = 0.25
verts_min = 10000
current_version = 30


key_source =  (((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}") & (dj.U("segment_id") & (minnie.OldBaylorSegmentCentroid() & "multiplicity<3").proj()) & minnie.NucleiSegmentsRun2())
               
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691134026921572,0,0.25
864691134141481226,0,0.25
864691134219060901,0,0.25
864691134221889045,0,0.25
864691134242673681,0,0.25
864691134252908410,0,0.25
864691134316523453,0,0.25
864691134351340112,0,0.25
864691134384033872,0,0.25
864691134391593458,0,0.25


In [24]:
len((dj.U("segment_id") & minnie.BaylorSegmentCentroid.proj()) & minnie.NucleiSegmentsRun2.proj())

65769

In [16]:
(minnie.BaylorSegmentCentroid & minnie.MultiSomaProofread2.proj()).delete()

AssertionError: Declaration context is not set

# Inspecting Decomposition

In [11]:
# minnie.Decomposition.drop()
# schema.external['decomposition'].delete(delete_external_files=True)

`microns_minnie65_02`.`__decomposition` (121 tuples)
Proceed? [yes, No]: yes


INFO - 2021-01-14 05:02:55,202 - table - Dropped table `microns_minnie65_02`.`__decomposition`


Tables dropped.  Restart kernel.


In [ ]:
minnie.Decomposition()

In [ ]:
(minnie.Decomposition() & "n_somas>1") & (minnie.NeuronGliaNuclei() & "n_glia_faces>0")

In [ ]:
((schema.jobs & "table_name='__decomposition'") & "status='error'")#.delete()

In [ ]:
curr_segment_id = 864691135462301853

In [ ]:
neuron_obj = (minnie.Decomposition & dict(segment_id=curr_segment_id)).fetch1("decomposition")

In [ ]:
import proofreading_utils as pru

In [ ]:
limb_results = pru.multi_soma_split_suggestions(neuron_obj)

In [ ]:
seperated_neurons_list= pru.split_neuron(neuron_obj = neuron_obj,
                limb_results=limb_results,
                plot_seperated_neurons=True)

In [ ]:
curr_limb = seperated_neurons_list[0][0]

In [ ]:
curr_limb.get_branch_names()

In [ ]:
nviz.plot_soma_limb_concept_network(neuron_obj)

In [ ]:
curr_limb = neuron_obj[0]

In [ ]:
 (74,61)

In [ ]:
import networkx as nx
nx.shortest_path(curr_limb.concept_network,74,61)

In [ ]:
import neuron_utils as nru

In [ ]:
points_to_show = nru.skeleton_points_along_path(curr_limb,branch_path=[74, 73, 36, 34, 28, 21, 32, 41, 54, 55, 58, 61],
                              skeletal_distance_per_coordinate = 5000)
points_to_show

In [ ]:
nviz.plot_objects(skeletons=[curr_limb.skeleton],
                 meshes=[neuron_obj["S0"].mesh,neuron_obj["S1"].mesh],
                 scatters=[points_to_show])

In [ ]:
nviz.visualize_neuron(neuron_obj,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict=dict(),
                     mesh_whole_neuron=True,
                      subtract_glia=False
                     )

In [ ]:
import neuron_visualizations as nviz
nviz.visualize_neuron(neuron_obj,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict="all",
                     mesh_whole_neuron=True,
                     subtract_glia=True)

In [ ]:
nviz.plot_soma_limb_concept_network(neuron_obj)

In [ ]:
pru.split_info_to_neuroglancer_link(segment_id=neuron_obj.segment_id,
                                   split_info=limb_results,
                                   output_type='server')

# Inspecting Split Suggestions

In [9]:
#minnie.NeuronSplitSuggestions.drop()

`microns_minnie65_02`.`__neuron_split_suggestions` (107 tuples)
Proceed? [yes, No]: yes


INFO - 2021-01-14 05:02:11,151 - table - Dropped table `microns_minnie65_02`.`__neuron_split_suggestions`


Tables dropped.  Restart kernel.


In [ ]:
minnie,schema = du.configure_minnie_vm()

In [ ]:
minnie.NeuronSplitSuggestions() & "segment_id=864691135847950686"

In [ ]:
(schema.jobs & "table_name='__neuron_split_suggestions'")# & "status='error'"

In [ ]:
limb_results = (minnie.NeuronSplitSuggestions() & dict(segment_id=864691135065033540)).fetch1("split_results")

In [ ]:
pru.split_info_to_neuroglancer_link(864691135065033540,split_info=limb_results,output_type="server")

In [4]:
du.create_suggested_splits_neuroglancer_spreadsheet(output_filename='allen_spreadsheet_transparent_paths.csv',
                                                   output_type="server")

segment_id = 864691134947393276
segment_id = 864691135012471798
segment_id = 864691135012571126
segment_id = 864691135012905110
segment_id = 864691135065018692
segment_id = 864691135065024068
segment_id = 864691135065032772
segment_id = 864691135065033540
segment_id = 864691135081745143
segment_id = 864691135155574372
segment_id = 864691135210051520
segment_id = 864691135274155493
segment_id = 864691135291777462
segment_id = 864691135291788214
segment_id = 864691135337764326
segment_id = 864691135341171909
segment_id = 864691135345134239
segment_id = 864691135345274271
segment_id = 864691135345276831
segment_id = 864691135348374487
segment_id = 864691135360133191
segment_id = 864691135373423816
segment_id = 864691135382504922
segment_id = 864691135382996203
segment_id = 864691135385192277
segment_id = 864691135385207893
segment_id = 864691135394014197
segment_id = 864691135403709422
segment_id = 864691135407333321
segment_id = 864691135428492848
segment_id = 864691135440515912
segment_

In [ ]:
debug

In [ ]:
neuron_obj.save_compressed_neuron?

In [ ]:
import datajoint_utils as du

In [ ]:
minnie,schema = du.configure_minnie_vm()

In [ ]:
import datajoint_utils as du

In [ ]:
minnie.Decom